In [23]:
#Entrega 2 Miguel Chaveinte

In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from scipy.stats import mode

In [25]:
#Leemos el dataset
data = pd.read_csv("C:/Users/migue/OneDrive/Escritorio/MINERIA/python-teodoro/practica1/seeds_dataset.txt", sep="\t")

In [26]:
data

,A,P,C,Len,Wid,Asy,Len2,Wheat
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


In [27]:
data.head()

,A,P,C,Len,Wid,Asy,Len2,Wheat
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [28]:
data.columns

Index(['A', 'P', 'C', 'Len', 'Wid', 'Asy', 'Len2', 'Wheat'], dtype='object')

In [29]:
#Separamos variable objetivo y estandarizamos datos
y=data.Wheat
y=np.array(y)
X = StandardScaler().fit_transform(data.iloc[:,:-1])
print(np.unique(y))
print('-----')
print(X)

[1 2 3]
-----
[[ 1.42097769e-01  2.15462437e-01  6.06017918e-05 ...  1.41701823e-01
  -9.86151745e-01 -3.83577423e-01]
 [ 1.11880257e-02  8.22375713e-03  4.28515270e-01 ...  1.97432229e-01
  -1.78816620e+00 -9.22013487e-01]
 [-1.92066576e-01 -3.60200562e-01  1.44238325e+00 ...  2.08047544e-01
  -6.67479334e-01 -1.18919199e+00]
 ...
 [-5.67570840e-01 -6.90247348e-01  7.33948301e-01 ... -7.06044846e-02
   3.07658816e+00 -7.18060432e-01]
 [-1.03608992e+00 -1.03564515e+00 -8.01701104e-01 ... -1.12152071e+00
  -6.81351965e-02 -7.42534799e-01]
 [-8.77620233e-01 -9.35863561e-01 -1.10234659e-01 ... -7.55292327e-01
   1.29122264e+00 -7.03783718e-01]]


In [30]:
df_describe = pd.DataFrame(X)
df_describe.describe()

,0,1,2,3,4,5,6
count,2.100000e+02,2.100000e+02,2.100000e+02,2.100000e+02,2.100000e+02,2.100000e+02,2.100000e+02
mean,-5.392512e-17,9.146123e-17,1.322091e-15,-2.182910e-15,-2.030122e-16,-3.679596e-16,-1.337554e-16
std,1.002389e+00,1.002389e+00,1.002389e+00,1.002389e+00,1.002389e+00,1.002389e+00,1.002389e+00
min,-1.466714e+00,-1.649686e+00,-2.668236e+00,-1.650501e+00,-1.668209e+00,-1.956769e+00,-1.813288e+00
25%,-8.879552e-01,-8.514330e-01,-5.980791e-01,-8.286816e-01,-8.349072e-01,-7.591477e-01,-7.404953e-01
50%,-1.696741e-01,-1.836639e-01,1.039927e-01,-2.376280e-01,-5.733534e-02,-6.746852e-02,-3.774588e-01
75%,8.465989e-01,8.870693e-01,7.116771e-01,7.945947e-01,8.044956e-01,7.123789e-01,9.563941e-01
max,2.181534e+00,2.065260e+00,2.006586e+00,2.367533e+00,2.055112e+00,3.170590e+00,2.328998e+00


In [61]:
min_clusters= np.unique(y).shape[0]  #el minimo es el numero de posibilidades de la clase
max_clusters=50
columnas=['single','complete','average','ward']
tabla = pd.DataFrame(index= list(range(min_clusters,max_clusters+1)),columns=columnas, data=np.zeros((max_clusters-min_clusters+1,4)),dtype=float)  #ccreamos tabla de resultados
tabla.columns.name='Linkage'
tabla.index.name='n_clusters'

for k in columnas:
    for c in range(min_clusters,max_clusters+1):
        clustering = AgglomerativeClustering(n_clusters=int(c),linkage=k) 
        clustering.fit_predict(X) #entrenamos con la carpeta correspondiente
        moda=-1 * np.ones(int(c),dtype='int') #moda por cada cluster
        for i in range(int(c)):
             moda[i] = mode(y[np.argwhere(clustering.labels_== i)],keepdims=False)[0].astype(int) #vemos clase mayoritaria en cada cluster
        y_test_predict = moda[clustering.labels_]  #clases asociadas por la moda en cada cluster predicho
        tabla.loc[c,k] = accuracy_score(y, y_test_predict) #comprobamos el accuracy de la real a la predicha
        #print(accuracy_score(y, y_test_predict))
        #print('-----')
    #print('******')
tabla

Linkage,single,complete,average,ward
n_clusters,,,,
3,0.347619,0.876190,0.880952,0.928571
4,0.352381,0.876190,0.880952,0.928571
5,0.357143,0.876190,0.880952,0.928571
6,0.357143,0.876190,0.880952,0.928571
7,0.361905,0.876190,0.890476,0.928571
8,0.361905,0.876190,0.890476,0.928571
9,0.366667,0.900000,0.890476,0.928571
10,0.376190,0.928571,0.890476,0.928571
11,0.380952,0.928571,0.890476,0.928571


In [82]:
print("El k optimo para AgglomerativeClustering en linkage single: ", tabla.single.argmax()+3, " con una tasa de aciertos: ", tabla.single.max())

El k optimo para AgglomerativeClustering en linkage single:  50  con una tasa de aciertos:  0.7857142857142857


In [83]:
print("El k optimo para AgglomerativeClustering en linkage single: ", tabla.complete.argmax()+3, " con una tasa de aciertos: ", tabla.complete.max())

El k optimo para AgglomerativeClustering en linkage single:  43  con una tasa de aciertos:  0.9571428571428572


In [84]:
print("El k optimo para AgglomerativeClustering en linkage single: ", tabla.average.argmax()+3, " con una tasa de aciertos: ", tabla.average.max())

El k optimo para AgglomerativeClustering en linkage single:  45  con una tasa de aciertos:  0.9571428571428572


In [85]:
print("El k optimo para AgglomerativeClustering en linkage single: ", tabla.ward.argmax()+3, " con una tasa de aciertos: ", tabla.ward.max())

El k optimo para AgglomerativeClustering en linkage single:  48  con una tasa de aciertos:  0.9619047619047619
